# Build json file from folders

## Create thumbnails in jpg & fix metadata. Series only contain one item

In [1]:
from PIL import Image
import json
import glob
import re
import pandas as pd

In [2]:
source_folder='C:/Users/Luis/Documents/luis/crypt/metatut/start/'
csv_file='C:/Users/Luis/Documents/luis/crypt/building.csv'
output_file=source_folder + "\\building3.json"
df=pd.read_csv(csv_file, sep=';')  

In [3]:
#sanity checks
sanity_parents={}
door_width=1
door_height=2

building={}
for i,row in df.iterrows():
    building[df["Gallery name"][i]]={}
    building[df["Gallery name"][i]]["geometry"]=df.loc[i, 'width': '#items E'].values.astype(int).tolist()
    building[df["Gallery name"][i]]["parent"]=df.loc[i, 'Parent']
    son_galleries=sum(df.Parent==df["Gallery name"][i])
    #if has son galleries
    if (son_galleries >0):
        for j, jrow in df.loc[df["Parent"]=="root"].iterrows():
            building[df["Gallery name"][i]][jrow["Gallery name"]]={}
            building[df["Gallery name"][i]][jrow["Gallery name"]]["resource"]=jrow["Gallery name"] +'.glb'
            building[df["Gallery name"][i]][jrow["Gallery name"]]["resource_type"]='gallery'
            building[df["Gallery name"][i]][jrow["Gallery name"]]["width"]= door_width
            building[df["Gallery name"][i]][jrow["Gallery name"]]["height"]= door_height
    else:
        for file_name in glob.glob(df["Folder"][i] + "/*.jpg"):
            im=Image.open(file_name)
            width, height = im.size
            as_ratio=height/width
            if (as_ratio<=1):
                width=1
                height=as_ratio
            else:
                width=1/as_ratio
                height=1

            name=re.search(r'.*\\(.*).jpg',file_name)[1]
            file_name_server=file_name.replace(source_folder, "./").replace('\\','/')

            building[df["Gallery name"][i]][name]= {"resource": file_name_server, "resource_type": 'image',"width": f'{width:.2f}', "height": f'{height:.2f}'}


    
with open(output_file, "w") as fp:
    json.dump(building,fp, indent=2)

In [90]:
file_name_server, source_folder

('C:/Users/Luis/Documents/luis/crypt/metatut/start/gallery2\\ChinstrapPenguin2.jpg',
 'C:/Users/Luis/Documents/luis/crypt/metatut/start/')

In [58]:
sum(df.Parent==df["Gallery name"][i])

2

In [34]:
list(df.loc[i, 'width': '#items E'].values)

[20, 10, 4, 1, 0, 1, 1]

In [7]:
source_folder='C:\\Users\\Luis\\Documents\\luis\\crypt\\metatut\\start\\'
gallery_folders=['','gallery1', 'gallery2']
gallery_names=['root','Antartida', 'Prueba']
#geometry width, length, height, itemsN,S,W,E
geometry=[[15,15,4,0,0,1,1],[20,40,4,7,7,11,11],[20,10,4,1,0,1,1]]

In [15]:
building={}


for i,gallery_folder in enumerate(gallery_folders):
    #create root gallery and 
    building[gallery_names[i]]={}
    building[gallery_names[i]]["geometry"]=geometry[i]
    if (gallery_names[i]=="root"):
        continue;

    for file_name in glob.glob(source_folder + gallery_folders[i] + "\\*.jpg"):
        im=Image.open(file_name)
        width, height = im.size
        as_ratio=height/width
        if (as_ratio<=1):
            width=1
            height=as_ratio
        else:
            width=1/as_ratio
            height=1
                
        name=re.search(r'.*\\(.*).jpg',file_name)[1]
        file_name_server=file_name.replace(source_folder, "./").replace("\\","/")
        
        building[gallery_names[i]][name]= {"file": file_name_server, "width": f'{width:.2f}', "height": f'{height:.2f}'}

        
with open(source_folder + "\\building2.json", "w") as fp:
    json.dump(building,fp, indent=2) 

In [30]:
print(json.dumps(building, indent=2))

TypeError: Object of type ndarray is not JSON serializable

In [8]:
len(building["Antartida"])

36